## Basic usage
Initialize an instance of Daemon, then call update to scan the cwd and build the Knowledge Graph.

In [29]:
from pathlib import Path
from ragdaemon.daemon import Daemon

daemon = Daemon(Path.cwd())
await daemon.update()
print(len(daemon.graph.nodes), "Nodes")

245 Nodes


## Search
Return be files, chunks of files (function, class, method or 'BASE'), or diffs (edits to a particular path and line range) sorted by embedding similarity.

Each search result includes (among other things):
1. `id`: Used as the id for nodes in the graph.
    - For files and chunks, a relative path to cwd. e.g. `mentat/config.py:Config`.
    - For diffs, its target location, e.g. `mentat/config.py:10-15`. 
2. `distance`: how related to query (lower is better)
3. `ref`: path + line numbers, used to load the document.
    - For files and chunks, relative path + lines, e.g. `mentat/config.py:10-15`
    - For diffs, the diff target ("DEFAULT" if none provided) + lines in diff, e.g. `DEFAULT:4-10`
4. `document`: The embedded content. Always f"{id}\n{content}" so the path / filename is also embedded, and there are no duplicates.
5. `checksum`: an md5 hash of the document, used as the chroma index.

In [30]:
results = daemon.search("raycaster", n=5)
for i, result in enumerate(results):
    if i == 0:
        fields = set(result.keys())
        print(f"Fields: {fields}")
    print(f"{i+1}. {result['id']}")

print("\nExample Result:")
for k, v in results[2].items():
    print(f"---{k}---\n{v}")

Fields: {'checksum', 'distance', 'ref', 'id', 'document', 'active', 'type'}
1. ragdaemon/static/js/three/raycaster.js:BASE
2. ragdaemon/static/js/three/raycaster.js
3. ragdaemon/static/js/three/raycaster.js:getClickTarget
4. ragdaemon/static/js/main.js:BASE
5. ragdaemon/static/js/three/renderer.js

Example Result:
---active---
True
---checksum---
ee0de86cb9a266dc9888b05f0342c4d0
---id---
ragdaemon/static/js/three/raycaster.js:getClickTarget
---ref---
ragdaemon/static/js/three/raycaster.js:10-28
---type---
chunk
---document---
ragdaemon/static/js/three/raycaster.js:10-28
10:function getClickTarget(event, dblclick=false) {
11:    // Calculate mouse position in normalized device coordinates
12:    // (-1 to +1) for both components
13:    mouse.x = (event.clientX / container.clientWidth) * 2 - 1;
14:    mouse.y = - (event.clientY / container.clientHeight) * 2 + 1;
15:
16:    // Update the picking ray with the camera and mouse position
17:    raycaster.setFromCamera(mouse, camera);
18:
19: 

## Get Context
Return a `ContextBuilder` with the search results pre-loaded.

The `ContextBuilder` contains
1. Methods to 'include' individual nodes: `add_ref(file_or_chunk_ref)` and `add_diff(diff_id)`. 
2. A `.render()` method to produce a single string to send to an LLM
3. All the logic related to how to sort, group and consolidate.

You can
- Call `daemon.get_context(query, auto_tokens).render()` straightaway for normal RAG
- Call `daemon.get_context("")` to return an empty context and add items manually
- Pass a ContextBuilder to get_context using the 'context_builder=' to add search results to existing context.

In [31]:
print(daemon.get_context("raycaster", auto_tokens=1000).render())

ragdaemon/static/js/three/raycaster.js (search-result)
1:import scene from './scene.js';
2:import camera from './camera.js';
3:
4:// Raycaster for mouse interaction
5:const raycaster = new THREE.Raycaster();
6:const mouse = new THREE.Vector2();
7:raycaster.far = camera.far * 10; // Set the raycaster's maximum distance to the camera's far plane
8:
9:const container = document.getElementById('scene-container');
10:function getClickTarget(event, dblclick=false) {
11:    // Calculate mouse position in normalized device coordinates
12:    // (-1 to +1) for both components
13:    mouse.x = (event.clientX / container.clientWidth) * 2 - 1;
14:    mouse.y = - (event.clientY / container.clientHeight) * 2 + 1;
15:
16:    // Update the picking ray with the camera and mouse position
17:    raycaster.setFromCamera(mouse, camera);
18:
19:    // Calculate objects intersecting the picking ray
20:    // Filter objects by those that have a toggleSelected method
21:    const clickables = dblclick
22:     